In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import neurokit2 as nk
import matplotlib.pyplot as plt


from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

import tensorflow as tf
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Flatten, Conv1D,TimeDistributed, Reshape, Activation, add, Layer, Concatenate, GRU, Bidirectional 
from tensorflow.keras.layers import MaxPooling1D, BatchNormalization, Dropout,Activation, GlobalAveragePooling1D

In [ ]:
# R-peak detection
def get_rpeak(signal, sampling_rate):
    _, rpeaks = nk.ecg_peaks(signal, sampling_rate=sampling_rate, show=False, method='neurokit')
    
    return rpeaks['ECG_R_Peaks']

# Segmentation Function

def load_BEAT_data(signal, r_peaks):
    signal_list = []
    for number in range(1,len(r_peaks)-1):
        signal_list.append(signal[r_peaks[number]-(int(left_sec*sampling_rate)) : r_peaks[number]+(int(right_sec*sampling_rate))])
    return np.array(signal_list)

def load_RHYTHM_data(data, rpeak, window):
    rhythm_data = []
    right = int(right_sec * sampling_rate)
    for i in range(1,len(rpeak)-1):
        if len(data[rpeak[i] + right : rpeak[i] + window + right]) == window:
            rhythm_data.append(data[rpeak[i] + right : rpeak[i]+right + window])

    return np.array(rhythm_data)

def shuffle_data(data, labels):
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    return data[indices], labels[indices]

def shuffle_data2(data1, data2, labels):
    # data1, data2, label을 입력받아 무작위로 섞고 (인덱스 위치는 동일하게) Return
    indices = np.arange(data1.shape[0])
    np.random.shuffle(indices)
    return data1[indices], data2[indices], labels[indices]

In [ ]:
def cal_class_weight(label):
    
    counts = np.bincount(label)
    
    amuse_num = counts[0]
    normal_num = counts[1]
    stress_num = counts[2]
    meditation_num = counts[3]
    total = amuse_num + normal_num + stress_num + meditation_num
    
    weight_amuse = (1/amuse_num) * (total/4.0)
    weight_normal = (1/normal_num) * (total/4.0)
    weight_stress = (1/stress_num) * (total/4.0)
    weight_meditation  = (1/meditation_num) * (total/4.0)
    class_weight = {0:weight_amuse, 1:weight_normal, 2:weight_stress, 3:weight_meditation}
    # print(total, class_weight)

    return class_weight

def Plot_lr_curve(history):
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy'] 
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    plt.figure(figsize=(20,5))
    plt.subplot(121)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend()
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')

    plt.subplot(122)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend()
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')

def Plot_confusion_matrix(true_d, pred, class_name):
    
    pred_label = []
    true_label = []
    
    for i in range(len(pred)):
        pred_label.append(np.argmax(pred[i]))
        
    for i in range(len(true_d)):
        true_label.append(np.argmax(true_d[i]))
    
    print(pred_label[0:10])
    # Accuracy
    accuracy = accuracy_score(true_label, pred_label)
    print('accuracy : ', np.round(accuracy,3))
    
    # Precision
    precision = precision_score(true_label, pred_label, average='macro')
    print('precision : ', np.round(precision,3))
    
    # Recall
    recall = recall_score(true_label, pred_label,average='macro')
    print('recall : ', np.round(recall,3))
    
    # F1 Score
    F1_score = f1_score(true_label, pred_label,average='macro')
    print('F1_score : ', np.round(F1_score,3))

    #clasification report
    report = classification_report(true_label, pred_label, target_names=class_name)#'Amuse',
    print(report)
    
    #confusion matrix
    confusion = confusion_matrix(true_label, pred_label)
    disp = ConfusionMatrixDisplay(confusion, display_labels=class_name)#'Amuse',
    disp.plot(cmap=plt.cm.Blues)
    plt.show()
    
    #plt.figure(figsize=(10,5))
    cmn = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=class_name, yticklabels=class_name)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show(block=False)

In [ ]:
import pickle5 as pickle

def load_data(db_path, segment, window=0):

    back_0_data, back_2_data, back_3_data, resting_data = [], [], [], []
    
    for i, p in enumerate(os.listdir(db_path)):
        
        with open(db_path+p,"rb") as fr:
            df = pickle.load(fr, encoding='bytes')
            
#         df = pd.read_pickle(db_path + p)
        # print(p)
        back_0 = np.concatenate((df['process Trial 1:0back'], df['process Trial 6:0back']))
        back_2 = np.concatenate((df['process Trial 2:2back'], df['process Trial 4:2back']))
        back_3 = np.concatenate((df['process Trial 3:3back'], df['process Trial 5:3back']))
        resting = df['Resting'].flatten()
    
        # print(back_0.shape, back_2.shape, back_3.shape, resting.shape)
    
        back_0_Rpeak = get_rpeak(back_0, sampling_rate)
        back_2_Rpeak = get_rpeak(back_2, sampling_rate)
        back_3_Rpeak = get_rpeak(back_3, sampling_rate)
        resting_Rpeak = get_rpeak(resting.flatten().flatten(), sampling_rate)
    
        if segment == 'beat':
            back_0_data.append(load_BEAT_data(back_0, back_0_Rpeak))
            back_2_data.append(load_BEAT_data(back_2, back_2_Rpeak))
            back_3_data.append(load_BEAT_data(back_3, back_3_Rpeak))
            resting_data.append(load_BEAT_data(resting, resting_Rpeak))
            
        else:
            back_0_data.append(load_RHYTHM_data(back_0, back_0_Rpeak, window))
            back_2_data.append(load_RHYTHM_data(back_2, back_2_Rpeak, window))
            back_3_data.append(load_RHYTHM_data(back_3, back_3_Rpeak, window))
            resting_data.append(load_RHYTHM_data(resting, resting_Rpeak, window))

    back_0_data = np.vstack(back_0_data)
    back_2_data = np.vstack(back_2_data)
    back_3_data = np.vstack(back_3_data)
    resting_data = np.vstack(resting_data)

    return back_0_data, back_2_data, back_3_data, resting_data


    return data, label

In [ ]:
left_sec = 0.24
right_sec = 0.4

sampling_rate=256

class_name = ['0_back', '2_back', '3_back', 'Resting']

In [ ]:
db_path = "D:/JH/Journal/MAUS_preprocessd/"
person = os.listdir(db_path)

In [ ]:
b_back_0, b_back_2, b_back_3, b_resting = load_data(db_path, 'beat')
# r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm')

In [ ]:
back_0_label = np.zeros(b_back_0.shape[0], np.int32)
back_2_label = np.ones(b_back_2.shape[0], np.int32) * 1
back_3_label = np.ones(b_back_3.shape[0], np.int32) * 2
resting_label = np.ones(b_resting.shape[0], np.int32) * 3

In [ ]:
beat_data = np.concatenate([b_back_0, b_back_2, b_back_3, b_resting])
# rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])

In [ ]:
Beat, Beat_label = shuffle_data(beat_data, label)

In [ ]:
train_b, test_b, train_label_y, test_label_y = train_test_split(Beat, Beat_label,
                                                            test_size=0.2,
                                                            stratify=Beat_label,
                                                            random_state=128,
                                                            shuffle=True)

train_b, val_b, train_label_y, val_label_y = train_test_split(train_b, train_label_y,
                                                         test_size=0.25,
                                                         stratify=train_label_y,
                                                         random_state=128,
                                                         shuffle=True)

print(train_b.shape, val_b.shape, test_b.shape)

In [ ]:
train_label = to_categorical(train_label_y)
val_label = to_categorical(val_label_y)
test_label = to_categorical(test_label_y)

# 1 layer 실험

In [ ]:
from keras.callbacks import EarlyStopping
# 조기 종료 
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)

# 가중치 초기화 
weight_init = HeNormal(seed= 128)

In [ ]:
save_model_path = "D:/JH/Journal/MAUS_Model/Beat_layer_setting/"

In [ ]:
def beat_model_1_layer(re_X_train_b):
    input_beat = Input(shape=(re_X_train_b,1), name = 'input_beat')

    conv1_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_beat)
    conv2_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B)
    bn1_B = BatchNormalization()(conv2_B)
    act_B = Activation('relu')(bn1_B)
    max1_B = MaxPooling1D(2,2)(act_B)
    
    GAP_B = GlobalAveragePooling1D()(max1_B)

    re_R = Reshape((1, GAP_B.shape[1]))(GAP_B)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_beat, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
class_weight = cal_class_weight(train_label_y)
print(class_weight)

MAUS_beat_model = beat_model_1_layer(train_b.shape[1])
MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = MAUS_beat_model.fit(train_b, train_label, 
                              validation_data=(val_b, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             ) 
scores = MAUS_beat_model.evaluate(test_b, test_label)
MAUS_beat_model.save(save_model_path + 'MAUS_beat_(622)_layer(1)_(' + str(np.round(scores[1]*100,2)) +').h5' )
Plot_lr_curve(history)
pred = MAUS_beat_model.predict(test_b)
Plot_confusion_matrix(test_label, pred, class_name)

# 2 layer 실험

In [ ]:
def beat_model_2_layer(re_X_train_b):
    input_beat = Input(shape=(re_X_train_b,1), name = 'input_beat')

    conv1_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_beat)
    conv2_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B)
    bn1_B = BatchNormalization()(conv2_B)
    act_B = Activation('relu')(bn1_B)
    max1_B = MaxPooling1D(2,2)(act_B)
    
    conv1_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B)
    conv2_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_1)
    bn1_B_1 = BatchNormalization()(conv2_B_1)
    act_B_1 = Activation('relu')(bn1_B_1)
    max1_B_1 = MaxPooling1D(2,2)(act_B_1)
    
    GAP_B = GlobalAveragePooling1D()(max1_B_1)

    re_R = Reshape((1, GAP_B.shape[1]))(GAP_B)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_beat, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
class_weight = cal_class_weight(train_label_y)
print(class_weight)

MAUS_beat_model = beat_model_2_layer(train_b.shape[1])
MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = MAUS_beat_model.fit(train_b, train_label, 
                              validation_data=(val_b, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             ) 
scores = MAUS_beat_model.evaluate(test_b, test_label)
MAUS_beat_model.save(save_model_path + 'MAUS_beat_(622)_layer(2)_(' + str(np.round(scores[1]*100,2)) +').h5' )
Plot_lr_curve(history)
pred = MAUS_beat_model.predict(test_b)
Plot_confusion_matrix(test_label, pred, class_name)

# 3 layer 실험

In [ ]:
def beat_model_3_layer(re_X_train_b):
    input_beat = Input(shape=(re_X_train_b,1), name = 'input_beat')

    conv1_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_beat)
    conv2_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B)
    bn1_B = BatchNormalization()(conv2_B)
    act_B = Activation('relu')(bn1_B)
    max1_B = MaxPooling1D(2,2)(act_B)
    
    conv1_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B)
    conv2_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_1)
    bn1_B_1 = BatchNormalization()(conv2_B_1)
    act_B_1 = Activation('relu')(bn1_B_1)
    max1_B_1 = MaxPooling1D(2,2)(act_B_1)
    
    conv1_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_1)
    conv2_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_2)
    bn1_B_2 = BatchNormalization()(conv2_B_2)
    act_B_2 = Activation('relu')(bn1_B_2)
    max1_B_2 = MaxPooling1D(2,2)(act_B_2)    
    
    GAP_B = GlobalAveragePooling1D()(max1_B_2)

    re_R = Reshape((1, GAP_B.shape[1]))(GAP_B)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_beat, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
class_weight = cal_class_weight(train_label_y)
print(class_weight)

MAUS_beat_model = beat_model_3_layer(train_b.shape[1])
MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = MAUS_beat_model.fit(train_b, train_label, 
                              validation_data=(val_b, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             ) 
scores = MAUS_beat_model.evaluate(test_b, test_label)
MAUS_beat_model.save(save_model_path + 'MAUS_beat_(622)_layer(3)_(' + str(np.round(scores[1]*100,2)) +').h5' )
Plot_lr_curve(history)
pred = MAUS_beat_model.predict(test_b)
Plot_confusion_matrix(test_label, pred, class_name)

# 4 layer 실험

In [ ]:
def beat_model_4_layer(re_X_train_b):
    input_beat = Input(shape=(re_X_train_b,1), name = 'input_beat')

    conv1_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_beat)
    conv2_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B)
    bn1_B = BatchNormalization()(conv2_B)
    act_B = Activation('relu')(bn1_B)
    max1_B = MaxPooling1D(2,2)(act_B)
    
    conv1_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B)
    conv2_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_1)
    bn1_B_1 = BatchNormalization()(conv2_B_1)
    act_B_1 = Activation('relu')(bn1_B_1)
    max1_B_1 = MaxPooling1D(2,2)(act_B_1)
    
    conv1_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_1)
    conv2_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_2)
    bn1_B_2 = BatchNormalization()(conv2_B_2)
    act_B_2 = Activation('relu')(bn1_B_2)
    max1_B_2 = MaxPooling1D(2,2)(act_B_2)    
    
    conv1_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_2)
    conv2_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_3)
    bn1_B_3 = BatchNormalization()(conv2_B_3)
    act_B_3 = Activation('relu')(bn1_B_3)
    max1_B_3 = MaxPooling1D(2,2)(act_B_3)    
    
    GAP_B = GlobalAveragePooling1D()(max1_B_3)

    re_R = Reshape((1, GAP_B.shape[1]))(GAP_B)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_beat, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
class_weight = cal_class_weight(train_label_y)
print(class_weight)

MAUS_beat_model = beat_model_4_layer(train_b.shape[1])
MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = MAUS_beat_model.fit(train_b, train_label, 
                              validation_data=(val_b, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             ) 
scores = MAUS_beat_model.evaluate(test_b, test_label)
MAUS_beat_model.save(save_model_path + 'MAUS_beat_(622)_layer(4)_(' + str(np.round(scores[1]*100,2)) +').h5' )
Plot_lr_curve(history)
pred = MAUS_beat_model.predict(test_b)
Plot_confusion_matrix(test_label, pred, class_name)

# 5 layer 실험

In [ ]:
def beat_model_5_layer(re_X_train_b):
    input_beat = Input(shape=(re_X_train_b,1), name = 'input_beat')

    conv1_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_beat)
    conv2_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B)
    bn1_B = BatchNormalization()(conv2_B)
    act_B = Activation('relu')(bn1_B)
    max1_B = MaxPooling1D(2,2)(act_B)
    
    conv1_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B)
    conv2_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_1)
    bn1_B_1 = BatchNormalization()(conv2_B_1)
    act_B_1 = Activation('relu')(bn1_B_1)
    max1_B_1 = MaxPooling1D(2,2)(act_B_1)
    
    conv1_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_1)
    conv2_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_2)
    bn1_B_2 = BatchNormalization()(conv2_B_2)
    act_B_2 = Activation('relu')(bn1_B_2)
    max1_B_2 = MaxPooling1D(2,2)(act_B_2)    
    
    conv1_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_2)
    conv2_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_3)
    bn1_B_3 = BatchNormalization()(conv2_B_3)
    act_B_3 = Activation('relu')(bn1_B_3)
    max1_B_3 = MaxPooling1D(2,2)(act_B_3)   
    
    conv1_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_3)
    conv2_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_4)
    bn1_B_4 = BatchNormalization()(conv2_B_4)
    act_B_4 = Activation('relu')(bn1_B_4)
    max1_B_4 = MaxPooling1D(2,2)(act_B_4) 
    
    GAP_B = GlobalAveragePooling1D()(max1_B_4)

    re_R = Reshape((1, GAP_B.shape[1]))(GAP_B)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_beat, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
class_weight = cal_class_weight(train_label_y)
print(class_weight)

MAUS_beat_model = beat_model_5_layer(train_b.shape[1])
MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = MAUS_beat_model.fit(train_b, train_label, 
                              validation_data=(val_b, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             ) 
scores = MAUS_beat_model.evaluate(test_b, test_label)
MAUS_beat_model.save(save_model_path + 'MAUS_beat_(622)_layer(5)_(' + str(np.round(scores[1]*100,2)) +').h5' )
Plot_lr_curve(history)
pred = MAUS_beat_model.predict(test_b)
Plot_confusion_matrix(test_label, pred, class_name)

# 6 layer 실험

In [ ]:
def beat_model_6_layer(re_X_train_b):
    input_beat = Input(shape=(re_X_train_b,1), name = 'input_beat')

    conv1_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_beat)
    conv2_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B)
    bn1_B = BatchNormalization()(conv2_B)
    act_B = Activation('relu')(bn1_B)
    max1_B = MaxPooling1D(2,2)(act_B)
    
    conv1_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B)
    conv2_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_1)
    bn1_B_1 = BatchNormalization()(conv2_B_1)
    act_B_1 = Activation('relu')(bn1_B_1)
    max1_B_1 = MaxPooling1D(2,2)(act_B_1)
    
    conv1_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_1)
    conv2_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_2)
    bn1_B_2 = BatchNormalization()(conv2_B_2)
    act_B_2 = Activation('relu')(bn1_B_2)
    max1_B_2 = MaxPooling1D(2,2)(act_B_2)    
    
    conv1_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_2)
    conv2_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_3)
    bn1_B_3 = BatchNormalization()(conv2_B_3)
    act_B_3 = Activation('relu')(bn1_B_3)
    max1_B_3 = MaxPooling1D(2,2)(act_B_3)   
    
    conv1_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_3)
    conv2_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_4)
    bn1_B_4 = BatchNormalization()(conv2_B_4)
    act_B_4 = Activation('relu')(bn1_B_4)
    max1_B_4 = MaxPooling1D(2,2)(act_B_4) 
    
    conv1_B_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_4)
    conv2_B_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_5)
    bn1_B_5 = BatchNormalization()(conv2_B_5)
    act_B_5 = Activation('relu')(bn1_B_5)
    max1_B_5 = MaxPooling1D(2,2)(act_B_5) 
    
    GAP_B = GlobalAveragePooling1D()(max1_B_5)

    re_R = Reshape((1, GAP_B.shape[1]))(GAP_B)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_beat, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
class_weight = cal_class_weight(train_label_y)
print(class_weight)

MAUS_beat_model = beat_model_6_layer(train_b.shape[1])
MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = MAUS_beat_model.fit(train_b, train_label, 
                              validation_data=(val_b, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             ) 
scores = MAUS_beat_model.evaluate(test_b, test_label)
MAUS_beat_model.save(save_model_path + 'MAUS_beat_(622)_layer(6)_(' + str(np.round(scores[1]*100,2)) +').h5' )
Plot_lr_curve(history)
pred = MAUS_beat_model.predict(test_b)
Plot_confusion_matrix(test_label, pred, class_name)

# layer 7

In [ ]:
def beat_model_7_layer(re_X_train_b):
    input_beat = Input(shape=(re_X_train_b,1), name = 'input_beat')

    conv1_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_beat)
    conv2_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B)
    bn1_B = BatchNormalization()(conv2_B)
    act_B = Activation('relu')(bn1_B)
    max1_B = MaxPooling1D(2,2)(act_B)
    
    conv1_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B)
    conv2_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_1)
    bn1_B_1 = BatchNormalization()(conv2_B_1)
    act_B_1 = Activation('relu')(bn1_B_1)
    max1_B_1 = MaxPooling1D(2,2)(act_B_1)
    
    conv1_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_1)
    conv2_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_2)
    bn1_B_2 = BatchNormalization()(conv2_B_2)
    act_B_2 = Activation('relu')(bn1_B_2)
    max1_B_2 = MaxPooling1D(2,2)(act_B_2)    
    
    conv1_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_2)
    conv2_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_3)
    bn1_B_3 = BatchNormalization()(conv2_B_3)
    act_B_3 = Activation('relu')(bn1_B_3)
    max1_B_3 = MaxPooling1D(2,2)(act_B_3)   
    
    conv1_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_3)
    conv2_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_4)
    bn1_B_4 = BatchNormalization()(conv2_B_4)
    act_B_4 = Activation('relu')(bn1_B_4)
    max1_B_4 = MaxPooling1D(2,2)(act_B_4) 
    
    conv1_B_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_4)
    conv2_B_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_5)
    bn1_B_5 = BatchNormalization()(conv2_B_5)
    act_B_5 = Activation('relu')(bn1_B_5)
    max1_B_5 = MaxPooling1D(2,2)(act_B_5) 
    
    conv1_B_6 = Conv1D(512, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_5)
    conv2_B_6 = Conv1D(512, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_6)
    bn1_B_6 = BatchNormalization()(conv2_B_6)
    act_B_6 = Activation('relu')(bn1_B_6)
    max1_B_6 = MaxPooling1D(2,2)(act_B_6)     
    
    GAP_B = GlobalAveragePooling1D()(max1_B_6)

    re_R = Reshape((1, GAP_B.shape[1]))(GAP_B)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_beat, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
class_weight = cal_class_weight(train_label_y)
print(class_weight)

MAUS_beat_model = beat_model_7_layer(train_b.shape[1])
MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = MAUS_beat_model.fit(train_b, train_label, 
                              validation_data=(val_b, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             ) 
scores = MAUS_beat_model.evaluate(test_b, test_label)
MAUS_beat_model.save(save_model_path + 'MAUS_beat_(622)_layer(7)_(' + str(np.round(scores[1]*100,2)) +').h5' )
Plot_lr_curve(history)
pred = MAUS_beat_model.predict(test_b)
Plot_confusion_matrix(test_label, pred, class_name)

# Beat 5-Fold 실험

### Train 80, Test 20, 학습률 0.00001 

In [ ]:
def beat_model_5_layer(re_X_train_b):
    input_beat = Input(shape=(re_X_train_b,1), name = 'input_beat')

    conv1_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_beat)
    conv2_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B)
    bn1_B = BatchNormalization()(conv2_B)
    act_B = Activation('relu')(bn1_B)
    max1_B = MaxPooling1D(2,2)(act_B)
    
    conv1_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B)
    conv2_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_1)
    bn1_B_1 = BatchNormalization()(conv2_B_1)
    act_B_1 = Activation('relu')(bn1_B_1)
    max1_B_1 = MaxPooling1D(2,2)(act_B_1)
    
    conv1_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_1)
    conv2_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_2)
    bn1_B_2 = BatchNormalization()(conv2_B_2)
    act_B_2 = Activation('relu')(bn1_B_2)
    max1_B_2 = MaxPooling1D(2,2)(act_B_2)    
    
    conv1_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_2)
    conv2_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_3)
    bn1_B_3 = BatchNormalization()(conv2_B_3)
    act_B_3 = Activation('relu')(bn1_B_3)
    max1_B_3 = MaxPooling1D(2,2)(act_B_3)   
    
    conv1_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_3)
    conv2_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_4)
    bn1_B_4 = BatchNormalization()(conv2_B_4)
    act_B_4 = Activation('relu')(bn1_B_4)
    max1_B_4 = MaxPooling1D(2,2)(act_B_4) 
    
    GAP_B = GlobalAveragePooling1D()(max1_B_4)

    re_R = Reshape((1, GAP_B.shape[1]))(GAP_B)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_beat, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
print(Beat.shape, Beat_label.shape)

In [ ]:
def scheuler(epoch, lr):
    if epoch < 41:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=128)

save_model_path = "D:/JH/Journal/MAUS_Model/Beat_5Fold/"
number = 1

for train_index, test_index in skf.split(Beat, Beat_label):
    print('=========================================__Fold__'+str(number)+'_______=================================')
    X_train_b, X_test_b = Beat[train_index], Beat[test_index]
    Y_train, Y_test = Beat_label[train_index], Beat_label[test_index]

    y_train = to_categorical(Y_train)
    y_test = to_categorical(Y_test)
    class_weight = cal_class_weight(Y_train)
    
    MAUS_beat_model = beat_model_5_layer(X_train_b.shape[1])
    callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)
    MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_beat_model.fit(X_train_b, y_train, 
                        validation_data=(X_test_b, y_test), 
                        epochs=50, 
                        batch_size=32,
                        callbacks=[callback_lr],
                        class_weight=class_weight
                       ) 

    
    scores = MAUS_beat_model.evaluate(X_test_b, y_test)
    Plot_lr_curve(history)
    pred = MAUS_beat_model.predict(X_test_b)
    Plot_confusion_matrix(y_test, pred, class_name)

    
    MAUS_beat_model.save(save_model_path + 'MAUS_beat_8020_Fold('  +str(number)+ ')_(' + str(np.round(scores[1]*100,2)) + ').h5')
    number+=1
    
    

# 학습률 0.0001 적용 
#### 0.00001에서 성능이 원활하게 나오지 않음

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=128)

save_model_path = "D:/JH/Journal/MAUS_Model/Beat_5Fold/"
number = 1

for train_index, test_index in skf.split(Beat, Beat_label):
    print('=========================================__Fold__'+str(number)+'_______=================================')
    X_train_b, X_test_b = Beat[train_index], Beat[test_index]
    Y_train, Y_test = Beat_label[train_index], Beat_label[test_index]

    y_train = to_categorical(Y_train)
    y_test = to_categorical(Y_test)
    class_weight = cal_class_weight(Y_train)
    
    MAUS_beat_model = beat_model_5_layer(X_train_b.shape[1])
    callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)
    MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_beat_model.fit(X_train_b, y_train, 
                        validation_data=(X_test_b, y_test), 
                        epochs=50, 
                        batch_size=32,
                        callbacks=[callback_lr],
                        class_weight=class_weight
                       ) 

    
    scores = MAUS_beat_model.evaluate(X_test_b, y_test)
    Plot_lr_curve(history)
    pred = MAUS_beat_model.predict(X_test_b)
    Plot_confusion_matrix(y_test, pred, class_name)

    
    MAUS_beat_model.save(save_model_path + 'MAUS_beat_8020_Fold('  +str(number)+ ')_(' + str(np.round(scores[1]*100,2)) + ')_0.0001_.h5')
    number+=1
    
    